In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
### ACTION ### CHANGE TO YOUR FILE PATH
df = pd.read_csv(r'C:\Users\aizhang\OneDrive - ANSYS, Inc\Desktop\Ad-Hoc Tasks\Chase\Migration Automation\Product Migration.csv')
df

In [ ]:
# quantity pivot table 
df['PRODUCT_TYPE_3'] = df['PRODUCT_TYPE_3'].fillna('')
type_qty_pivot = df.pivot_table(index=['DESIGNATED_SITE_ANSYS_ACCOUNT_NUMBER','ACCOUNT_NAME'], columns=['PRODUCT_TYPE_2','PRODUCT_TYPE_3'], values = 'QUANTITY', aggfunc='sum', fill_value='')
type_qty_pivot

In [ ]:
### ACTION ### UPDATE THE COLUMN HEADER NAMES BASED ON YOUR ANALYSIS
column_order = [
    ('Enterprise',''), ('Premium',''),('Pro',''),('Solver',''),
    ('Add-on','Far Infrared Extension'),
    ('Add-on','HUD Design & Analysis'),
    ('Add-on','Optical Design Optimizer'),
    ('Add-on','Optical Part Design'),
    ('Add-on','Optical Sensor Test')
]

type_qty_pivot = type_qty_pivot.loc[:, column_order]
type_qty_pivot

In [ ]:
type_qty_pivot = pd.DataFrame(type_qty_pivot)
type_qty_pivot

In [ ]:
# flatten the columns 
type_qty_pivot.columns = [col[1] if col[0] == 'Add-on' else col[0] for col in type_qty_pivot.columns]

In [ ]:
type_qty_pivot

In [ ]:
# config qty
rows_qty = []

for index, row in type_qty_pivot.iterrows():
    filtered_row = [value for value in row if value != '']

    rows_qty.append(filtered_row)

for row in rows_qty:
    print(row)

In [ ]:
### ACTION ### INPUT ORIGINAL PRODUCT AND ITS PRICE TO CALCULATE ORIGINAL ACV
product_price = {
    'Enterprise': 101450,
    'Premium': 43960,
    'Pro': 20290,
    'Solver': 16710,
    'Far Infrared Extension': 27050,
    'HUD Design & Analysis': 27050,
    'Optical Design Optimizer': 6760,
    'Optical Part Design': 27050,
    'Optical Sensor Test': 27050
}

In [ ]:
# pivot table for acv calculation
acv_pivot = type_qty_pivot
for product, price in product_price.items():
    acv_pivot[product] = acv_pivot[product] * price

acv_pivot

In [ ]:
type_qty_pivot

In [ ]:
# acv list 
rows_acv = []

for index, row in acv_pivot.iterrows():
    filtered_row = [value for value in row if value != '']

    rows_acv.append(filtered_row)

for row in rows_acv:
    print(row)

In [ ]:
# total quantity list 
total_acv = [sum(acv) for acv in rows_acv]
total_acv

In [ ]:
# config pivot table 
type_qty_pivot2 = type_qty_pivot.applymap(lambda x: '' if x == 0 else None)
for col in type_qty_pivot.columns:
    type_qty_pivot2[col] = type_qty_pivot[col].apply(lambda x: col if x != '' else '')

type_qty_pivot2

In [ ]:
# config list 
rows = []

for index, row in type_qty_pivot2.iterrows():
    filtered_row = [value for value in row if value != '']

    rows.append(filtered_row)

for row in rows:
    print(row)

In [ ]:
# total quantity list 
total_qty = [sum(qty) for qty in rows_qty]
total_qty

In [ ]:
# account list 
account_numbers = type_qty_pivot.index.get_level_values('DESIGNATED_SITE_ANSYS_ACCOUNT_NUMBER').tolist()
account_names = type_qty_pivot.index.get_level_values('ACCOUNT_NAME').tolist()
account_numbers

In [ ]:
account_names

In [ ]:
# combine the output 
combine_data = []
for account_id, account_name, config, qty, total_qty, total_acv in zip(account_numbers, account_names, rows, rows_qty, total_qty, total_acv):
    combine_data.append({
        'Account Number': account_id,
        'Account Name': account_name,
        'License Configuration': config,
        'License Quantity': qty,
        'Total License Quantity': total_qty,
        'ACV1': total_acv
    })

combine_data

In [ ]:
len(combine_data)

In [ ]:
output1 = pd.DataFrame(combine_data)
output1['License Configuration'] = output1['License Configuration'].apply(lambda x: ', '.join(map(str,x)))
output1['License Quantity'] = output1['License Quantity'].apply(lambda x: ', '.join(map(str,x)))
output1['Migration Path'] = None 
output1['Migration License Quantity'] = None 
output1['Upsell Migration Path'] = None 
output1['Upsell Migration Quantity'] = None 

output1

In [ ]:
migration_path = output1.groupby('License Configuration').agg(
    Account_Count = ('Account Number','count'),
    Total_num_of_licenses = ('Total License Quantity','sum')
).reset_index()

migration_path['Migration/Renewal Products'] = None
migration_path['Upsell Opportunity'] = None
migration_path

In [ ]:
### ACTION ### UPDATE FILE OUTPUT PATH 
with pd.ExcelWriter(r'C:\Users\aizhang\OneDrive - ANSYS, Inc\Desktop\Ad-Hoc Tasks\Chase\Migration Automation\output1.xlsx') as writer: 
    output1.to_excel(writer, sheet_name='Migration-renewal Quantity', index=False)
    migration_path.to_excel(writer, sheet_name='Migration-renewal pathways', index=False)

print(f"Data has been written to your file path")